In [1]:
from googleapiclient.discovery import build
from google_auth_oauthlib.flow import InstalledAppFlow
from google.auth.transport.requests import Request

import urllib.parse as p
import re
import os
import pickle
from google.auth.exceptions import RefreshError
import logging
import pandas as pd 

logging.basicConfig(level=logging.WARNING)
logger = logging.getLogger(__name__)

SCOPES = ["https://www.googleapis.com/auth/youtube.force-ssl"]

In [2]:
def youtube_authenticate():
    os.environ["OAUTHLIB_INSECURE_TRANSPORT"] = "1"
    api_service_name = "youtube"
    api_version = "v3"
    client_secrets_file = r'D:\2A\2A\mods207\code_secret_client_1.json'
    creds = None
    # the file token.pickle stores the user's access and refresh tokens, and is
    # created automatically when the authorization flow completes for the first time
    if os.path.exists("token_1.pickle"):
        with open("token_1.pickle", "rb") as token:
            creds = pickle.load(token)
    # if there are no (valid) credentials availablle, let the user log in.
    if creds and creds.refresh_token:
        try:
            creds.refresh(Request())
        except RefreshError:
            logger.error("Credentials could not be refreshed, possibly the authorization was revoked by the user.")
            os.unlink('token_1.pickle')
            return
    else:
        flow = InstalledAppFlow.from_client_secrets_file(
            client_secrets_file, SCOPES)
        #flow.redirect_uri = "http://localhost:52684/"
        creds = flow.run_local_server(port=52683)

    # Save the credentials for the next run
    with open('token_1.pickle', 'wb') as token:
        pickle.dump(creds, token)
            
            


    return build(api_service_name, api_version, credentials=creds)

# authenticate to YouTube API
youtube = youtube_authenticate()

Please visit this URL to authorize this application: https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=196350673502-d24c6bqeqkaojep9d72pf42lib1garrr.apps.googleusercontent.com&redirect_uri=http%3A%2F%2Flocalhost%3A52683%2F&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fyoutube.force-ssl&state=BPp0grYgWrnaokHRXr7xJlubr9cayI&access_type=offline


MismatchingStateError: (mismatching_state) CSRF Warning! State not equal in request and response.

In [3]:
channels_ids = pd.read_csv('channel.csv')

In [4]:
channels_ids

,channels
0,UCWeg2Pkate69NFdBeuRFTAw
1,UCyWqModMQlbIo8274Wh_ZsQ
2,UCww2zZWg4Cf5xcRKG-ThmXQ
3,UCNEKMkg_DG8eAyR1BNWsSvw
4,UCCB1Byx5yTbLpQaV-rlfmtA
...,...
92,UCfih6kPJCpzWmtCFtlpYK6A
93,UCdTyuXgmJkG_O8_75eqej-w
94,UCVRcmhHcXhKVMbupBbrEtUA
95,UCLI5qZqBkuMkZ54EK6Zi6AA


In [5]:
def get_channel_videos(youtube, **kwargs):
    return youtube.search().list(
        **kwargs
    ).execute()


def get_channel_details(youtube, **kwargs):
    return youtube.channels().list(
        part="statistics,snippet,contentDetails",
        **kwargs
    ).execute()

In [6]:
def get_video_id_by_url(url):
    """
    Return the Video ID from the video `url`
    """
    # split URL parts
    parsed_url = p.urlparse(url)
    # get the video ID by parsing the query of the URL
    video_id = p.parse_qs(parsed_url.query).get("v")
    if video_id:
        return video_id[0]
    else:
        raise Exception(f"Wasn't able to parse video URL: {url}")

In [41]:
channel_titles = []
channel_descriptions = []
channel_creation_dates = []
channel_video_counts = []
channel_subscriber_counts = []
channel_view_counts = []

for channel_id in channel_ids :
    
    response = get_channel_details(youtube, id=channel_id)
    # extract channel infos
    snippet = response["items"][0]["snippet"]
    statistics = response["items"][0]["statistics"]
    channel_description = snippet["description"]
    channel_creation_date = snippet["publishedAt"]
    channel_title = snippet["title"]
    channel_subscriber_count = statistics["subscriberCount"]
    channel_video_count = statistics["videoCount"]
    channel_view_count  = statistics["viewCount"]
    
    channel_titles.append(channel_title)
    channel_descriptions.append(channel_description)
    channel_creation_dates.append(channel_creation_date)
    channel_video_counts.append(channel_video_count)
    channel_subscriber_counts.append(channel_subscriber_count)
    channel_view_counts.append(channel_view_count)
    
    print(f"""
    Title: {channel_title}
    Published At: {channel_creation_date}
    Description: {channel_description}
    Number of videos: {channel_video_count}
    Number of subscribers: {channel_subscriber_count}
    Total views: {channel_view_count}
    """)


    Title: SQUEEZIE
    Published At: 2011-01-09T01:56:21Z
    Description: clique sur une vidéo nan??
    Number of videos: 1456
    Number of subscribers: 17000000
    Total views: 9041537469
    

    Title: Cyprien
    Published At: 2007-02-25T22:14:05Z
    Description: Je fais des vidéos, en plus tu peux même les regarder ! La nature fait bien les choses...
    Number of videos: 202
    Number of subscribers: 14300000
    Total views: 2965558646
    

    Title: NORMAN FAIT DES VIDÉOS
    Published At: 2011-01-04T00:45:42Z
    Description: Je fais des vidéos YouTube depuis... toujours 🤷🏼‍♂️
    Number of videos: 201
    Number of subscribers: 12000000
    Total views: 2662182898
    

    Title: OGGY
    Published At: 2014-10-24T15:26:43Z
    Description: Welcome to the official Oggy Youtube channel!

Subscribe and get new episodes of Oggy and the Cockroaches every week! Watch tons of gags and discover lots of exclusive material: assortment of episodes sorted by theme, a grab-bag

KeyError: 'subscriberCount'

In [7]:
n_pages = 200
# counting number of videos grabbed
n_videos = 0
next_page_token = None
video_ids = []
video_urls = []
for i in range(n_pages):
    params = {
        'part': 'snippet',
        'q': '',
        'channelId': channels_ids['channels'][1],
        'type': 'video',
    }
    if next_page_token:
        params['pageToken'] = next_page_token
    res = get_channel_videos(youtube, **params)
    channel_videos = res.get("items")
    for video in channel_videos:
        n_videos += 1
        video_id = video["id"]["videoId"]
        video_ids.append(video_id)
        # easily construct video URL by its ID
        video_url = f"https://www.youtube.com/watch?v={video_id}"
        video_urls.append(video_url)
        
        print(f"Video URL: {video_url}")
        print("nb videos", n_videos)
        print("="*40)
    print("*"*100)
    # if there is a next page, then add it to our parameters
    # to proceed to the next page
    if "nextPageToken" in res:
        next_page_token = res["nextPageToken"]

NameError: name 'youtube' is not defined

In [ ]:
channel = pd.DataFrame()
channel['vid_url'] = video_urls
channel['vid_id'] = video_id
path = f"channel_0_{channels_ids['channels'][0]}.csv"
channel.to_csv(path, index=False)